# N-Queens Problem Exploration with Python and an Evolutionary Algorithm Approach

In [32]:
import random, itertools

## this is a heading 2


Please see https://github.com/aiml-satx/n-queen-evolutionary-algo/blob/master/README.md for background.

A chromosome will be in the form of a tuple with N members representing the location of a queen on each row of an N-row square grid. We reduce our problem space by having only one queen per row and a unique integer for each item in the tuple, which represents one queen per column.

Let's start with a way to randomly generate a layout of queens, and a utility function to print the layout:

In [33]:
def gen_random(N):
    queen_layout = list(range(N))
    random.shuffle(queen_layout)
    return tuple(queen_layout)

In [3]:
def print_queen_layout(queen_layout, ascii_out=False):
    N = len(queen_layout)
    if ascii_out:
        qn = ' x '
        sq = ' o '
    else:
        qn = u'\u265B'
        sq = u'\u2B1C'
    print("\n".join(sq*i+qn+sq*(N-i-1) for i in queen_layout))

In [4]:
# Let's see how this looks:
queen_layout = gen_random(8)
print(queen_layout)
print_queen_layout(queen_layout, ascii_out=False)
print_queen_layout(queen_layout, ascii_out=True)

(3, 5, 2, 7, 0, 1, 4, 6)
⬜⬜⬜♛⬜⬜⬜⬜
⬜⬜⬜⬜⬜♛⬜⬜
⬜⬜♛⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜♛
♛⬜⬜⬜⬜⬜⬜⬜
⬜♛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜♛⬜⬜⬜
⬜⬜⬜⬜⬜⬜♛⬜
 o  o  o  x  o  o  o  o 
 o  o  o  o  o  x  o  o 
 o  o  x  o  o  o  o  o 
 o  o  o  o  o  o  o  x 
 x  o  o  o  o  o  o  o 
 o  x  o  o  o  o  o  o 
 o  o  o  o  x  o  o  o 
 o  o  o  o  o  o  x  o 


Next define a fitness function to evaluate the fitness chromosome (representing a single member of a population).

In [5]:
def nqueens_fitness(queen_layout):
    """ scores a particular layout of n-queens (where each diagonal with a collision deducts a point from 
            a maximum score of N (with no double-counting.))
    
        Scoring is from this screenshot from Mark Stumbris:
        https://aimlofsanantonio.slack.com/files/UHBE7TYSH/FHB8HC82Y/screen_shot_2019-03-26_at_10.45.17_pm.png
        
        This is really a cool way to score, because you're taking the sum of the value at a particular
            index and its index number (queen_layout[i] + i), which should be unique if there are no other
            queens on that diagonal.  So the number of unique sums (or differences for the other diagonals)
            should be N in the case of a solution.
            
        Inputs:
            queen_layout: n-tuple of integers 
    """
    N = len(queen_layout)
    cols = range(N)
    if N==len(set(queen_layout[i]+i for i in cols))==len(set(queen_layout[i]-i for i in cols)):
        return N
    else:
        return ((N-len(set(queen_layout[i] + i for i in cols))) + (N-len(set(queen_layout[i]-i for i in cols))))/2
        

Next we should have a way to randomly mutate an individual chromosome in order to introduce variety in the population.

In [6]:
def gen_mutate(queen_layout, strength=0.5):
    N=len(queen_layout)
    queen_layout=list(queen_layout)
    strength=strength/N
    for i in range(N):
        if random.random() <= strength:
            # swap value with another item by randomly choosing another item's index (could be itself)
            j = random.randint(0,N-1)
            queen_layout[i], queen_layout[j] = queen_layout[j], queen_layout[i]
    return tuple(queen_layout)


We should now define a function to "breed" two members of a population.  In our case we will randomly snip a section from one parent and drop in the items from the other parent that are not included in the random section, but in the order of the second parent. (I know this sounds confusing, so please see this paper for an illustration: 
https://arxiv.org/pdf/1802.02006.pdf near the end of section 3.)

In [7]:
def gen_crossover(parent_1, parent_2, mutate_prob=0.5):
    
    N = len(parent_1)
    start_idx = random.randint(0, N-2)
    end_idx = (random.randint(start_idx+1, (N)))
    parent_1 = list(parent_1)
    parent_2 = list(parent_2)
    
    child_1 = parent_1.copy()
    child_2 = parent_2.copy()
    
    for itm in parent_1[start_idx:end_idx]:
        child_2.remove(itm)
        
    for itm in parent_2[start_idx:end_idx]:
        child_1.remove(itm)
    
    parent_temp = parent_1.copy()
    for x in reversed(parent_2[start_idx:end_idx]): child_1.insert(start_idx,x)
    for x in reversed(parent_temp[start_idx:end_idx]): child_2.insert(start_idx,x)
    
    # add a mutation step, before returning two children
    child_1 = gen_mutate(child_1)
    child_2 = gen_mutate(child_2)
    return child_1, child_2

In [8]:
# Try it out:
parent_1 = gen_random(8)
parent_2 = gen_random(8)
child_1, child_2 = gen_crossover(parent_1, parent_2)
print("Parent 1: %s, Parent 2: %s" % (parent_1, parent_2))
print("Child 1: %s, Child 2: %s" % (child_1, child_2))

Parent 1: (2, 1, 0, 5, 4, 7, 3, 6), Parent 2: (3, 6, 0, 1, 7, 2, 5, 4)
Child 1: (3, 6, 0, 2, 1, 5, 4, 7), Child 2: (2, 1, 0, 3, 6, 7, 5, 4)


Now I think we're ready to start up a population, evaluate, select, crossover, mutate and iterate.  We'll do this with a slightly larger grid (10x10) and see what happens:

In [9]:
N=10   # number of rows and columns
pop_size = 10  # population size
max_generations = 5000  # maximum number of generations to create

# Initialize population
population = [gen_random(N) for i in range(pop_size)]

counter=0

while True:
    counter+=1
    # Evaluate population
    pop_scores = []
    for itm in population:
        pop_scores.append(nqueens_fitness(itm))
    
    if N in pop_scores:
        print("Solution found at generation %s: " % (counter))
        print ("Population: %s\nScores: %s" % (population, pop_scores))
        print(print_queen_layout(population[pop_scores.index(N)]))
        break
        
    #print("Scores: %s, avg: %s" % (pop_scores, sum(pop_scores)/pop_size))

    # Select parents
    chosen = random.choices(population, weights=pop_scores, k=int(N/2))
    
    # Combine parents
    population=[]
    chosen_N = len(chosen)
    for i in range(chosen_N-1):
        if i==chosen_N-2:
            j=0
        elif i==chosen_N-1:
            j=1
        else:
            j=i+1
        k=j+1
        
        population.extend(list(gen_crossover(chosen[i], chosen[j])))
        population.extend(list(gen_crossover(chosen[i], chosen[k])))
        
    #print("Population: %s, counter: %s" % (population, counter))
    
    # Set a limit to the number of iterations
    if counter>max_generations:
        print("No solution found within %s generations." % max_generations)
        break

Solution found at generation 1346: 
Population: [(6, 8, 0, 5, 2, 1, 3, 9, 7, 4), (6, 7, 0, 8, 9, 1, 3, 5, 2, 4), (6, 2, 0, 5, 8, 9, 1, 3, 7, 4), (1, 5, 0, 8, 2, 7, 9, 3, 6, 4), (6, 7, 0, 5, 2, 9, 3, 8, 1, 4), (2, 0, 4, 7, 9, 1, 3, 8, 6, 5), (7, 6, 5, 0, 2, 1, 4, 8, 3, 9), (6, 7, 0, 5, 2, 1, 3, 8, 4, 9), (1, 2, 0, 5, 6, 3, 7, 8, 4, 9), (7, 0, 5, 2, 1, 8, 9, 3, 6, 4), (7, 0, 5, 6, 1, 3, 2, 9, 8, 4), (0, 6, 1, 5, 7, 9, 3, 8, 2, 4), (6, 7, 0, 5, 2, 1, 3, 8, 4, 9), (6, 5, 0, 8, 9, 1, 3, 7, 2, 4), (6, 7, 0, 5, 2, 1, 3, 8, 4, 9), (6, 7, 0, 3, 2, 1, 5, 8, 4, 9)]
Scores: [2.5, 2.5, 1.0, 1.5, 2.5, 2.0, 2.0, 3.0, 3.0, 3.0, 2.0, 10, 3.0, 2.0, 3.0, 3.5]
♛⬜⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜♛⬜⬜⬜
⬜♛⬜⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜♛⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜♛⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜⬜♛
⬜⬜⬜♛⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜⬜⬜♛⬜
⬜⬜♛⬜⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜♛⬜⬜⬜⬜⬜
None


As an exercise for the reader, perhaps we should run the above cell in a loop and get the average number of generations until a solution.  I just ran it repeatedly and got the following generation counts: (1998, 1302, 337, 2381, 343, 63, 161, 1072).  So, it looks like the number of individual fitness evaluations (population x generations) are on the order of 10,000 on average.

In [10]:
# what is the average number of evaluations.
generations = [1998, 1302, 337, 2381, 343, 63, 161, 1072]
avg_gens = sum(generations)/len(generations)

# Now multiply by the population in each generation to get the number of evaluations:
print(avg_gens * 10)

9571.25


We should compare this approach with the brute force approach.  I'll use the aforementioned Mark Stumbris code and count the number of iterations to get to the first solution:

In [11]:
N=10 # I wouldn't recommend any N>12 or so.
sol=0
cols=range(N)
counter = 0
for combo in itertools.permutations(cols):
    counter+=1
    if N==len(set(combo[i]+i for i in cols))==len(set(combo[i]-i for i in cols)):
        sol += 1
        print('Solution %s: %s \n' % (sol, combo))
        print_queen_layout(combo, ascii_out=True)
        print("%s itertations" % (counter,))
        break

Solution 1: (0, 2, 5, 7, 9, 4, 8, 1, 3, 6) 

 x  o  o  o  o  o  o  o  o  o 
 o  o  x  o  o  o  o  o  o  o 
 o  o  o  o  o  x  o  o  o  o 
 o  o  o  o  o  o  o  x  o  o 
 o  o  o  o  o  o  o  o  o  x 
 o  o  o  o  x  o  o  o  o  o 
 o  o  o  o  o  o  o  o  x  o 
 o  x  o  o  o  o  o  o  o  o 
 o  o  o  x  o  o  o  o  o  o 
 o  o  o  o  o  o  x  o  o  o 
58987 itertations


### Conclusion:

So, the EA method averages under 10,000 evaluations to get a solution, where the brute force approach finds its first solution at around 59,000 evaluations.

### Extra Innings...

The following code adds some visualization -- and some dependencies.  So, I've just included it at the bottom of the notebook for folks who don't want to add the bokeh package to their python install/environment.

In [12]:
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

Make the code above into a function.

In [13]:
def queen_solver(N, pop_size, max_generations=200000):

    # Initialize population
    population = [gen_random(N) for i in range(pop_size)]

    counter=0
    avg_scores = []
    
    while True:
        counter+=1
        # Evaluate population
        pop_scores = []
        for itm in population:
            pop_scores.append(nqueens_fitness(itm))

        avg_scores.append(sum(pop_scores)/len(pop_scores))
        
        if N in pop_scores:
            #print("Solution found at generation %s: " % (counter))
            #print ("Population: %s\nScores: %s" % (population, pop_scores))
            #print(print_queen_layout(population[pop_scores.index(N)]))
            return population[pop_scores.index(N)], counter, avg_scores

        # Select parents
        chosen = random.choices(population, weights=pop_scores, k=int(N/2))

        # Combine parents
        population=[]
        chosen_N = len(chosen)
        for i in range(chosen_N-1):
            if i==chosen_N-2:
                j=0
            elif i==chosen_N-1:
                j=1
            else:
                j=i+1
            k=j+1

            population.extend(list(gen_crossover(chosen[i], chosen[j])))
            population.extend(list(gen_crossover(chosen[i], chosen[k])))

        #print("Population: %s, counter: %s" % (population, counter))

        # Set a limit to the number of iterations
        if counter>max_generations:
            print("No solution found within %s generations." % max_generations)
            return None, counter, avg_scores

In [14]:
gen_counts = {}
grid_sizes = [6, 7, 8, 9, 10, 11, 12, 13, 14]
for i in grid_sizes:
    print(f'\n{i}x{i} grid.\n iteration: ')
    for j in range(20):
        print(f'{j}', end=' ')
        solution, count, avg_scores = queen_solver(i, 10)
        if i in gen_counts:
            gen_counts[i].append((count, avg_scores))
        else:
            gen_counts[i]=[(count, avg_scores)]


6x6 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
7x7 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
8x8 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
9x9 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
10x10 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
11x11 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
12x12 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
13x13 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
14x14 grid.
 iteration: 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

In [27]:
# Plot the stats on evaluation counts for solutions of different size grids

grid_size=[]
gen_avgs = []
gen_mins = []
gen_maxes = []

for itm, vals in gen_counts.items():
    grid_size.append(itm)
    counts = np.array([k[0] for k in vals])
    gen_avgs.append(counts.mean()*10)
    gen_mins.append(counts.min()*10)
    gen_maxes.append(counts.max()*10)

In [28]:
f = figure()
f.line(grid_size, gen_avgs, line_color='cornflowerblue', legend='ea mean evaluations')
f.line(grid_size, gen_mins, line_color='green', legend='ea minimum evaluations')
f.line(grid_size, gen_maxes, line_color='red', legend='ea maximum evaluations')
f.legend.location = "top_left"

In [29]:
show(f)

In [20]:
# How do things look for the brute force approach?

def nqueen_brute_force(grid_size):
    sol=0
    cols=range(grid_size)
    counter = 0
    for combo in itertools.permutations(cols):
        counter+=1
        if grid_size==len(set(combo[i]+i for i in cols))==len(set(combo[i]-i for i in cols)):
            sol += 1
            print('Solution for %sx%s grid in %s evaluations.\n' % (grid_size, grid_size, counter))
            print_queen_layout(combo, ascii_out=True)
            #print("%s itertations" % (counter))
            return (combo, counter)


In [21]:
N = [6,7,8,9,10,11,12,13] # I wouldn't recommend any N>12 or so.

solutions = {}
     
for grid_size in N:
    solutions[grid_size] = nqueen_brute_force(grid_size)

Solution for 6x6 grid in 187 evaluations.

 o  x  o  o  o  o 
 o  o  o  x  o  o 
 o  o  o  o  o  x 
 x  o  o  o  o  o 
 o  o  x  o  o  o 
 o  o  o  o  x  o 
Solution for 7x7 grid in 187 evaluations.

 x  o  o  o  o  o  o 
 o  o  x  o  o  o  o 
 o  o  o  o  x  o  o 
 o  o  o  o  o  o  x 
 o  x  o  o  o  o  o 
 o  o  o  x  o  o  o 
 o  o  o  o  o  x  o 
Solution for 8x8 grid in 2843 evaluations.

 x  o  o  o  o  o  o  o 
 o  o  o  o  x  o  o  o 
 o  o  o  o  o  o  o  x 
 o  o  o  o  o  x  o  o 
 o  o  x  o  o  o  o  o 
 o  o  o  o  o  o  x  o 
 o  x  o  o  o  o  o  o 
 o  o  o  x  o  o  o  o 
Solution for 9x9 grid in 7686 evaluations.

 x  o  o  o  o  o  o  o  o 
 o  o  x  o  o  o  o  o  o 
 o  o  o  o  o  x  o  o  o 
 o  o  o  o  o  o  o  x  o 
 o  x  o  o  o  o  o  o  o 
 o  o  o  x  o  o  o  o  o 
 o  o  o  o  o  o  o  o  x 
 o  o  o  o  o  o  x  o  o 
 o  o  o  o  x  o  o  o  o 
Solution for 10x10 grid in 58987 evaluations.

 x  o  o  o  o  o  o  o  o  o 
 o  o  x  o  o  o  o  o  o  

In [22]:
solutions

{6: ((1, 3, 5, 0, 2, 4), 187),
 7: ((0, 2, 4, 6, 1, 3, 5), 187),
 8: ((0, 4, 7, 5, 2, 6, 1, 3), 2843),
 9: ((0, 2, 5, 7, 1, 3, 8, 6, 4), 7686),
 10: ((0, 2, 5, 7, 9, 4, 8, 1, 3, 6), 58987),
 11: ((0, 2, 4, 6, 8, 10, 1, 3, 5, 7, 9), 462121),
 12: ((0, 2, 4, 7, 9, 11, 5, 10, 1, 6, 8, 3), 4545700),
 13: ((0, 2, 4, 1, 8, 11, 9, 12, 3, 5, 7, 10, 6), 47369404)}

In [30]:
f.line(N, [i[1][1] for i in solutions.items()], line_color='darkgray', legend='brute force')

GlyphRenderer(id='1702', ...)

In [31]:
show(f)